 # Gray-Scott Model for Reaction-Diffusion in 3-D
 
We implement the Gray-Scott model in three dimensions. This involves adding another dimension to the 2-D grid used in the 2 dimensional case. This extra dimension, however, adds difficulties in computation and visualisation. 

The mathematical model for the three dimensional case is the same as the two dimensional case, where the concentrations of A and B are updated following the system of equations:

$$A' = A + (D_{A}\nabla^{2}A - AB^{2} + f(1-A))\Delta t$$
and
$$B' = B + (D_{b}\nabla^{2}B + AB^{2} - (k+f)B)\Delta t$$

The diffusion operator $\nabla^2$ needs to be modified to three dimensions. This can be done using a 3D kernel. Here, we use an unbiased kernel with a centre value of -1 and surrounding values of 1/26. Other kernels can be used for different results.

The numpy arrays are then converted to VTK files for and visualised with [Paraview](https://www.paraview.org/).



<video controls src="animation.mp4" />

In [50]:
# import relevant libraries
import numpy as np
from scipy import ndimage
from pyevtk.hl import imageToVTK
import numpy as np
import os, re, math
from pyevtk.vtk import VtkGroup


In [52]:
# f, k are feed and kill rate, stoptime is the stop time, width is the size of the 3D output
# where the size is equal to width x width x width
def diffusion_3D(f, k, stoptime, width):
    # da and db = diffusion rates, dt = time step, t = starting time
    da,db,dt,t = .24,.12,1,0

    # 3x3x3 kernel for convolution
    kernel = np.array([[[1/26, 1/26, 1/26],[1/26, 1/26, 1/26], [1/26, 1/26, 1/26]],
                     [[1/26, 1/26, 1/26],[1/26, -1, 1/26], [1/26, 1/26, 1/26]],
                     [[1/26, 1/26, 1/26],[1/26, 1/26, 1/26], [1/26, 1/26, 1/26]]])

    # initialise the grid, A=1,  B=0, and C=0
    A = np.ones([width,width,width])
    B = np.zeros([width,width,width])
    C = np.zeros([width,width,width])

    # add small seed area
    x = int(width/2)
    B[x:x+1+int(math.ceil(width/64)),x:x+1+int(math.ceil(width/64)),x:x+1+int(math.ceil(width/64))] = 1

    # update the grid using the diffusion equations
    count = 1
    while t<stoptime:
        A += (da*ndimage.convolve(A,kernel,mode='reflect',cval=0.0) - A*B**2 + f*(1-A))*dt
        B += (db*ndimage.convolve(B,kernel,mode='reflect',cval=0.0) + A*B**2 - (k+f)*B)*dt
        if t%50 == 0 or t==0:
            np.save('./images3d/A/'+str(count)+'.npy', A)
            np.save('./images3d/B/'+str(count)+'.npy', B)
            count+=1
        t +=  dt
    return A, B

# sorts list numerically
def numericalSort(list):
    def key(value):
        numbers = re.compile(r'(\d+)')
        parts = numbers.split(value)
        parts[1::2] = map(int, parts[1::2])
        return parts
    return sorted(list, key=key)

# convert numpy arrays to VTK and add them to PVD group
def convert_to_vtk_group(directory, data_type, output_directory):
    filelist = numericalSort(os.listdir(directory))
    g = VtkGroup("./group")
    time = 0
    if data_type == 'image':
        for file in filelist:
            temp = np.load(directory+'/'+file)
            imageToVTK(output_directory + '/' + file[:-4], cellData = {'data': temp})
            g.addFile(filepath = output_directory + '/' + file[:-4]+'.vti', sim_time = time)
            time += 1
    g.save()

In [103]:
# compute the diffusion arrays and convert to VTK
A, B = diffusion_3D(.032, .065, 10000, 128)
directory = './images3d/A'
output_directory = './vtk_files'
convert_to_vtk_group(directory, 'image', output_directory)